In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna
from optuna import Trial
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [68]:
filtered_df = pd.read_csv("aggday.csv" )
filtered_df  ['data_user']= filtered_df  ['data_user'].astype(str)
filtered_df  ['Source'] =filtered_df  ['consumer_device'] + '_'+'data_user'+'_'+filtered_df  ['data_user']
# Convertir 'Date' en datetime
filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])
# Créer toutes les combinaisons possibles entre Source et Date
#sources = filtered_df["Source"].unique()
#date_range = pd.date_range(start=filtered_df["Date"].min(), end=filtered_df["Date"].max())
# MultiIndex avec toutes les combinaisons Source x Date
#full_index = pd.MultiIndex.from_product([sources, date_range], names=["Source", "Date"])
#full_df = pd.DataFrame(index=full_index).reset_index()
# Fusion avec le dataset original
#filtered_df = full_df.merge(filtered_df, on=["Source", "Date"], how="left")
## Remplacer les valeurs NaN par 0
#filtered_df.fillna(0, inplace=True)

In [69]:
### CHANGE NAME OF FOLDER AND FILES TO CORRESPOND ####
climate_df = pd.read_excel(r"C:\Users\User\Desktop\ZINDI\Climate Data\Kalam Climate Data.xlsx")# Convert to datetime
filtered_df["Date"] = pd.to_datetime(filtered_df ["Date"])
climate_df["Date Time"] = pd.to_datetime(climate_df["Date Time"])

# Aggregate climate data to daily level
climate_daily = climate_df.groupby(climate_df["Date Time"].dt.date).agg({
    "Temperature (°C)": "mean",
    "Dewpoint Temperature (°C)": "mean",
    "U Wind Component (m/s)": "mean",
    "V Wind Component (m/s)": "mean",
    "Total Precipitation (mm)": "sum",
    "Snowfall (mm)": "sum",
    "Snow Cover (%)": "mean",
}).reset_index()

# Convert 'Date' column in climate_daily to datetime format
climate_daily.rename(columns={"Date Time": "Date"}, inplace=True)
climate_daily["Date"] = pd.to_datetime(climate_daily["Date"])  # Ensure datetime64[ns]

# Merge with complete_data
complete_data = filtered_df .merge(climate_daily, on="Date", how="left")

In [70]:
# Convert 'Date' to datetime format if not already
complete_data["Date"] = pd.to_datetime(complete_data["Date"])

# Extraire les composantes temporelles
complete_data["year"] = complete_data["Date"].dt.year  # Gardé tel quel
complete_data["is_weekend"] = (complete_data["Date"].dt.dayofweek >= 5).astype(int)  # 1 si Sam/Dim, sinon 0

# Transformation cyclique avec cosinus et sinus
complete_data["month_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.month / 12)
complete_data["week_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.isocalendar().week / 52)
complete_data["day_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.day / 31)
complete_data["day_of_week_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.dayofweek / 7)


In [71]:
complete_data[['consumer_device', 'data_user']] = complete_data['Source'].str.extract(r'(consumer_device_\d+)_data_user_(\d+)')

# clustering


In [72]:
df2=complete_data.copy()

In [85]:
data=complete_data  [['Source', 'Date', 'consumer_device',
        'Temperature (°C)', 
       'Dewpoint Temperature (°C)', 'U Wind Component (m/s)',
       'V Wind Component (m/s)', 'Total Precipitation (mm)', 'Snowfall (mm)',
       'Snow Cover (%)', 'year', 'is_weekend', 'month_cos', 'week_cos',
       'day_cos', 'day_of_week_cos' ,'kwh']].copy()

In [88]:


# Sélectionner les colonnes pertinentes pour le clustering
columns_for_clustering = ['v_red', 'power_factor', 'v_blue', 'v_yellow', 'kwh']
X = df2[columns_for_clustering]

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Appliquer K-means clustering
kmeans = KMeans(n_clusters=5)  # On peut ajuster le nombre de clusters
df2['Cluster'] = kmeans.fit_predict(X_scaled)

# Extraire les sources uniques
unique_sources = df2['Source'].unique()

# Créer un DataFrame avec les points uniques des sources
df_unique_sources = df2[df2['Source'].isin(unique_sources)]

# Afficher les résultats
print(df_unique_sources[['Source', 'Cluster']].head())


                          Source  Cluster
0  consumer_device_1_data_user_1        1
1  consumer_device_1_data_user_1        1
2  consumer_device_1_data_user_1        1
3  consumer_device_1_data_user_1        1
4  consumer_device_1_data_user_1        1


In [89]:
from sklearn.metrics import davies_bouldin_score

db_score = davies_bouldin_score(X_scaled, df2['Cluster'])
print(f"Davies-Bouldin Index: {db_score}")


Davies-Bouldin Index: 0.7358122009905308


In [90]:
# Sélectionner la colonne 'Cluster' de df_unique_sources
df_cluster = df_unique_sources[['Source', 'Date', 'Cluster']]

# Convertir la colonne 'Date' des deux DataFrames au même format pour assurer une bonne fusion
data['Date'] = pd.to_datetime(data['Date'])
df_cluster['Date'] = pd.to_datetime(df_cluster['Date'])

# Fusionner 'data' et 'df_cluster' selon 'Source' et 'Date'
merged_data = pd.merge(data, df_cluster, on=['Source', 'Date'], how='left')

C:\Users\User\AppData\Local\Temp\ipykernel_14940\2738764802.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster['Date'] = pd.to_datetime(df_cluster['Date'])


In [91]:
data=merged_data.copy()
data['Cluster']= data  ['Cluster'].astype(str)

In [92]:
ss = pd.read_csv("Data/SampleSubmission.csv")
ss[['date', 'consumer_device', 'data_user']] = ss['ID'].str.extract(r'(\d{4}-\d{2}-\d{2})_(consumer_device_\d+)_data_user_(\d+)')

In [93]:
ss['Date']=ss['date']
# Convert 'Date' to datetime format if not already
ss["Date"] = pd.to_datetime(ss["Date"])

# Extraire les composantes temporelles
ss["year"] = ss["Date"].dt.year  # Gardé tel quel
ss["is_weekend"] = (ss["Date"].dt.dayofweek >= 5).astype(int)  # 1 si Sam/Dim, sinon 0

# Transformation cyclique avec cosinus et sinus
ss["month_cos"] = np.cos(2 * np.pi * ss["Date"].dt.month / 12)
ss["week_cos"] = np.cos(2 * np.pi * ss["Date"].dt.isocalendar().week / 52)
ss["day_cos"] = np.cos(2 * np.pi * ss["Date"].dt.day / 31)
ss["day_of_week_cos"] = np.cos(2 * np.pi * ss["Date"].dt.dayofweek / 7)
### CHANGE NAME OF FOLDER AND FILES TO CORRESPOND ####
climate_df = pd.read_excel(r"C:\Users\User\Desktop\ZINDI\Climate Data\Kalam Climate Data.xlsx")# Convert to datetime
ss["Date"] = pd.to_datetime(filtered_df ["Date"])
climate_df["Date Time"] = pd.to_datetime(climate_df["Date Time"])

# Aggregate climate data to daily level
climate_daily = climate_df.groupby(climate_df["Date Time"].dt.date).agg({
    "Temperature (°C)": "mean",
    "Dewpoint Temperature (°C)": "mean",
    "U Wind Component (m/s)": "mean",
    "V Wind Component (m/s)": "mean",
    "Total Precipitation (mm)": "sum",
    "Snowfall (mm)": "sum",
    "Snow Cover (%)": "mean",
}).reset_index()

# Convert 'Date' column in climate_daily to datetime format
climate_daily.rename(columns={"Date Time": "Date"}, inplace=True)
climate_daily["Date"] = pd.to_datetime(climate_daily["Date"])  # Ensure datetime64[ns]

# Merge with complete_data
ss = ss.merge(climate_daily, on="Date", how="left")
ss ['data_user']= ss ['data_user'].astype(str)
ss ['Source'] =ss  ['consumer_device'] + '_'+'data_user'+'_'+ss  ['data_user']

In [94]:
ss.drop(columns=["ID",'date','data_user','kwh'], inplace=True)


In [95]:
# Sélectionner la colonne 'Cluster' de df_unique_sources
df_cluster = df_unique_sources[['Source', 'Date', 'Cluster']]

# Convertir la colonne 'Date' des deux DataFrames au même format pour assurer une bonne fusion
data['Date'] = pd.to_datetime(data['Date'])
df_cluster['Date'] = pd.to_datetime(df_cluster['Date'])

# Fusionner 'data' et 'df_cluster' selon 'Source' et 'Date'
ss = pd.merge(ss, df_cluster, on=['Source', 'Date'], how='left')

C:\Users\User\AppData\Local\Temp\ipykernel_14940\405763222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster['Date'] = pd.to_datetime(df_cluster['Date'])


In [44]:
result = pd.concat([data, ss], axis=0).reset_index(drop=True)
data=result.iloc[:198832-6014,:]
ss=result.iloc[-6014:,:]

In [96]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

# Assurer que 'Date' est au format datetime
data["Date"] = pd.to_datetime(data["Date"])

# Trier par Source puis par Date
data = data.sort_values(["Source", "Date"])

# Sélectionner les 30 derniers jours de chaque Source
test_indices = data.groupby("Source")["Date"].transform(lambda x: x >= x.max() - pd.Timedelta(days=30))
train_data = data[~test_indices]
test_data = data[test_indices]
# Sélectionner les 30 derniers jours de chaque Source
#test_indices = data.groupby("Source")["Date"].transform(lambda x: x >= x.max() - pd.Timedelta(days=0))
#train_data = data[~test_indices]
#test_data = data[test_indices]

# Séparer les features et la cible
X_train, y_train = train_data.drop(columns=["kwh", "Date"]), train_data["kwh"]
X_test, y_test = test_data.drop(columns=["kwh", "Date"]), test_data["kwh"]

# Définir les colonnes catégorielles
cat_features =  ["consumer_device", "Source", "Cluster"] 

# Initialiser et entraîner CatBoost
model = CatBoostRegressor(iterations=1000, learning_rate=0.01, cat_features=cat_features, verbose=200)
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

# Prédictions
y_pred = model.predict(X_test)


0:	learn: 2.2837270	test: 10.7021054	best: 10.7021054 (0)	total: 67ms	remaining: 1m 6s
200:	learn: 1.4470767	test: 7.3446261	best: 7.3446261 (200)	total: 10.5s	remaining: 41.8s
400:	learn: 1.3754894	test: 6.8120153	best: 6.8120153 (400)	total: 21.4s	remaining: 32s
600:	learn: 1.3492992	test: 6.7329202	best: 6.7329202 (600)	total: 33.9s	remaining: 22.5s
800:	learn: 1.3326039	test: 6.6936875	best: 6.6932853 (798)	total: 46.6s	remaining: 11.6s
999:	learn: 1.3165294	test: 6.6596787	best: 6.6596044 (998)	total: 59.2s	remaining: 0us

bestTest = 6.65960442
bestIteration = 998

Shrink model to first 999 iterations.


In [61]:
X_ss= ss.drop(columns=["Date",'kwh'])


In [62]:
y_predss = model.predict(X_ss)


In [64]:
rer = pd.DataFrame(y_predss)
rer.rename(columns={0: 'kwh'}, inplace=True)

te = pd.read_csv("Data/SampleSubmission.csv")
te['kwh'] = rer['kwh']
te.to_csv('subcat666.csv', index=False)
